In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data
import torchvision

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.utils import save_image

import pandas as pd
import numpy as np
import scipy.io
import skimage.io

from PIL import Image, ImageFilter

# Dataset Class

In [2]:
# input size => (N, 3, 256, 320)

class PartAffordanceDataset(Dataset):
    """Part Affordance Dataset"""
    
    def __init__(self, csv_file, transform=None):
        super().__init__()
        
        self.image_class_path = pd.read_csv(csv_file)
        self.transform = transform
        
    def __len__(self):
        return len(self.image_class_path)
    
    def __getitem__(self, idx):
        image_path = self.image_class_path.iloc[idx, 0]
        class_path = self.image_class_path.iloc[idx, 1]
        image = skimage.io.imread(image_path) # read as numpy array
        cls = scipy.io.loadmat(class_path)["gt_label"]
        
        sample = {'image': image, 'class': cls}
        
        if self.transform:
            sample = self.transform(sample)
            
        return sample

In [3]:
def crop_center_numpy(array, crop_height, crop_weight):
    h, w = array.shape
    return array[h//2 - crop_height//2: h//2 + crop_height//2,
                 w//2 - crop_weight//2: w//2 + crop_weight//2
                ]

In [4]:
def crop_center_pil_image(pil_img, crop_width, crop_height):
    img_width, img_height = pil_img.size
    return pil_img.crop(((img_width - crop_width) // 2,
                         (img_height - crop_height) // 2,
                         (img_width + crop_width) // 2,
                         (img_height + crop_height) // 2))

In [5]:
class CenterCrop(object):
    def __call__(self, sample):
        image, cls = sample['image'], sample['class']
        
        image = Image.fromarray(np.uint8(image))
        
        image = crop_center_pil_image(image, 320, 256)
        cls = crop_center_numpy(cls, 256, 320)
        
        image = np.asarray(image)
        
        return {'image': image, 'class': cls}

In [6]:
class ToTensor(object):
    def __call__(self, sample):
        image, cls = sample['image'], sample['class']
        
        image = image.transpose((2, 0, 1))
        # cls = cls.transpose((2, 0, 1))
        return {'image': torch.from_numpy(image).float(), 
                'class': torch.from_numpy(cls).long()}

In [7]:
# mean = 0
# std = 0
# n = 0

# for sample in data_laoder:
#     img = sample['image']   
#     img = img.view(len(img), 3, -1)
#     mean += img.mean(2).sum(0)
#     std += img.std(2).sum(0)
#     n += len(img)
    
# mean /= n
# std /= n

In [8]:
mean=[55.8630, 59.9099, 91.7419]
std=[31.6852, 29.8496, 19.0835]

In [9]:
class Normalize(object):
    def __call__(self, sample):
        image, cls = sample['image'], sample['class']
        
        image = transforms.functional.normalize(image, mean, std)
        
        return {'image': image, 'class': cls}

In [10]:
train_data = PartAffordanceDataset('train.csv',
                                transform=transforms.Compose([
                                    CenterCrop(),
                                    # OneHot(), # don't need to convert class into one-hot when calculating CrossEntropy Loss
                                    ToTensor(),
                                    Normalize()
                                ]))

In [11]:
test_data = PartAffordanceDataset('test.csv',
                                transform=transforms.Compose([
                                    CenterCrop(),
                                    ToTensor(),
                                    Normalize()
                                ]))

In [12]:
train_loader = DataLoader(train_data, batch_size=10, shuffle=True, num_workers=4)
test_loader = DataLoader(test_data, batch_size=10, shuffle=False, num_workers=4)

### class weight

In [13]:
# cnt_dict = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0, 6:0, 7:0}

# for sample in data_laoder:
#     img = sample['class'].numpy()
    
#     num, cnt = np.unique(img, return_counts=True)
    
#     for n, c in zip(num, cnt):
#         cnt_dict[n] += c

cnt_dict  
 0: 2078085712,  
 1: 34078992,  
 2: 15921090,  
 3: 12433420,  
 4: 38473752,  
 5: 6773528,  
 6: 9273826,  
 7: 20102080  

In [14]:
class_num = torch.tensor([2078085712, 34078992, 15921090, 12433420, 
                          38473752, 6773528, 9273826, 20102080])

total = class_num.sum().item()
print(total)

2215142400


In [15]:
frequency = class_num.float() / total
median = torch.median(frequency)

In [16]:
class_weight = median / frequency

In [17]:
class_weight

tensor([0.0077, 0.4672, 1.0000, 1.2805, 0.4138, 2.3505, 1.7168, 0.7920])

# Define Model

In [13]:
class DeconvBn_2(nn.Module):
    """ Deconvolution(stride=2) => Batch Normilization """
    
    def __init__(self, in_channel, out_channel):
        super().__init__()
        
        self.deconv = nn.ConvTranspose2d(in_channel, out_channel, kernel_size=2, stride=2, bias=False)
        self.bn = nn.BatchNorm2d(out_channel)
        
    def forward(self, x):
        return self.bn(self.deconv(x))

In [14]:
class DeconvBn_8(nn.Module):
    """ Deconvolution(stride=8) => Batch Normilization """
    
    def __init__(self, in_channel, out_channel):
        super().__init__()
        
        self.deconv = nn.ConvTranspose2d(in_channel, out_channel, kernel_size=8, stride=8, bias=False)
        self.bn = nn.BatchNorm2d(out_channel)
        
    def forward(self, x):
        return self.bn(self.deconv(x))

In [15]:
class FCN8s(nn.Module):
    """ Fully Convolutional Network"""
    
    def __init__(self, in_channel, n_classes):
        super().__init__()
        vgg =torchvision.models.vgg16_bn(pretrained=True).features
        
        # confirm the architecture of vgg16 by "print(vgg)"
        
        self.pool3 = vgg[:24]
        self.pool4 = vgg[24:34]
        self.pool5 = vgg[34:]
        
        self.deconv_bn1 = DeconvBn_2(512, 512)
        self.deconv_bn2 = DeconvBn_2(512, 256)
        self.deconv_bn3 = DeconvBn_8(256, n_classes)
        
    def forward(self, x):
        # vgg16
        x3 = self.pool3(x)     # output size => (N, 256, H/8, W/8)
        x4 = self.pool4(x3)    # output size => (N, 512, H/16, W/16)
        x5 = self.pool5(x4)    # output size => (N, 512, H/32, W/32)
        
        score = self.deconv_bn1(x5)
        score = self.deconv_bn2(x4 + score)
        score = self.deconv_bn3(x3 + score)
        
        return score

# Training

In [16]:
from tensorboardX import SummaryWriter
import tqdm

In [17]:
# reference

# SMOOTH = 1e-6

# def iou_pytorch(outputs: torch.Tensor, labels: torch.Tensor):
#     # You can comment out this line if you are passing tensors of equal shape
#     # But if you are passing output from UNet or something it will most probably
#     # be with the BATCH x 1 x H x W shape
#     outputs = outputs.squeeze(1)  # BATCH x 1 x H x W => BATCH x H x W
    
#     intersection = (outputs & labels).float().sum((1, 2))  # Will be zero if Truth=0 or Prediction=0
#     union = (outputs | labels).float().sum((1, 2))         # Will be zzero if both are 0
    
#     iou = (intersection + SMOOTH) / (union + SMOOTH)  # We smooth our devision to avoid 0/0
    
#     thresholded = torch.clamp(20 * (iou - 0.5), 0, 10).ceil() / 10  # This is equal to comparing with thresolds
    
#     return thresholded  # Or thresholded.mean() if you are interested in average across the batch

In [18]:
def eval_model(model, test_loader, device='cpu'):
    model.eval()
    
    intersection = torch.zeros(8)   # the dataset has 8 classes including background
    union = torch.zeros(8)
    
    for sample in test_loader:
        x, y = sample['image'], sample['class']
        
        x = x.to(device)
        y = y.to(device)
        
        with torch.no_grad():
            _, ypred = model(x).max(1)    # y_pred.shape => (N, 240, 320)
        
        for i in range(8):
            y_i = (y == i)           
            ypred_i = (ypred == i)   

            inter = (y_i.byte() & ypred_i.byte()).float().sum().to('cpu')
            intersection[i] += inter
            union[i] += (y_i.float().sum() + ypred_i.float().sum()).to('cpu') - inter
    
    """ iou[i] is the IoU of class i """
    iou = intersection / union
    
    return iou

In [19]:
def train_model(model, train_loader, test_loader, optimizer_cls=optim.Adadelta, 
                criterion=nn.CrossEntropyLoss(), max_epoch=200, device='cpu', writer=None):
    
    model.to(device)
    
    train_losses = []
    val_iou = []
    mean_iou = []
    best_mean_iou = 0.0
    
    optimizer = optimizer_cls(model.parameters())
    
    for epoch in range(max_epoch):
        model.train()
        running_loss = 0.0
        
        for i, sample in tqdm.tqdm(enumerate(train_loader), total=len(train_loader)):
            optimizer.zero_grad()
            
            x, y = sample['image'], sample['class']
            
            x = x.to(device)
            y = y.to(device)

            h = model(x)
            loss = criterion(h, y)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()

        train_losses.append(running_loss / i)
        
        val_iou.append(eval_model(model, test_loader, device).to('cpu').float())
        mean_iou.append(val_iou[-1].mean().item())
        
        if best_mean_iou < mean_iou[-1]:
            best_mean_iou = mean_iou[-1]
            torch.save(model.state_dict(), "./FCN_results_without_class_weight/best_mean_iou_model.prm")
        
        if writer is not None:
            writer.add_scalar("train_loss", train_losses[-1], epoch)
            writer.add_scalar("mean_IoU", mean_iou[-1], epoch)
            writer.add_scalars("class_IoU", {'iou of class 0': val_iou[-1][0],
                                           'iou of class 1': val_iou[-1][1],
                                           'iou of class 2': val_iou[-1][2],
                                           'iou of class 3': val_iou[-1][3],
                                           'iou of class 4': val_iou[-1][4],
                                           'iou of class 5': val_iou[-1][5],
                                           'iou of class 6': val_iou[-1][6],
                                           'iou of class 7': val_iou[-1][7]}, epoch)
            
        print(epoch, train_losses[-1], mean_iou[-1])
        
    torch.save(model.state_dict(), "./FCN_results_without_class_weight/final_model.prm")

In [20]:
model = FCN8s(3, 8)
writer = SummaryWriter("./FCN_results_without_class_weight/")
train_model(model, train_loader, test_loader, criterion=nn.CrossEntropyLoss(), device="cuda", writer=writer)

100%|██████████| 2360/2360 [07:29<00:00,  5.25it/s]


0 0.17716328388150274 0.8809050917625427


100%|██████████| 2360/2360 [07:29<00:00,  5.25it/s]


1 0.01992074077954455 0.8966249823570251


100%|██████████| 2360/2360 [07:28<00:00,  5.26it/s]


...

197 0.000336306526104755 0.924422025680542


100%|██████████| 2360/2360 [07:34<00:00,  5.19it/s]


198 0.00032150784773930146 0.9243425130844116


100%|██████████| 2360/2360 [07:35<00:00,  5.18it/s]


199 0.00031502902253114214 0.9245156049728394


In [26]:
colors = torch.tensor([[0, 0, 0],         # class 0 'background'  black
                       [255, 0, 0],       # class 1 'grasp'       red
                       [255, 255, 0],     # class 2 'cut'         yellow
                       [0, 255, 0],       # class 3 'scoop'       green
                       [0, 255, 255],     # class 4 'contain'     sky blue
                       [0, 0, 255],       # class 5 'pound'       blue
                       [255, 0, 255],     # class 6 'support'     purple
                       [255, 255, 255]    # class 7 'wrap grasp'  white
                      ])

In [27]:
def class_to_mask(cls):
    
    mask = colors[cls].transpose(1, 2).transpose(1, 3)
    
    return mask

In [28]:
def predict(model, sample, device='cpu'):
    model.eval()
    model.to(device)
    
    x, y = sample['image'], sample['class']
    
    x = x.to(device)
    y = y.to(device)

    with torch.no_grad():
        _, y_pred = model(x).max(1)    # y_pred.shape => (N, 240, 320)
    
    true_mask = class_to_mask(y).to('cpu')
    pred_mask = class_to_mask(y_pred).to('cpu')
    
    save_image(true_mask, "./FCN_results_without_class_weight/true_mask_with_FCN.jpg")
    save_image(pred_mask, "./FCN_results_without_class_weight/pred_mask_with_FCN.jpg")

In [31]:
trained_model = FCN8s(3, 8)
trained_model.load_state_dict(torch.load("./FCN_results_without_class_weight/best_mean_iou_model.prm"))

eval_data = PartAffordanceDataset('eval.csv',
                                transform=transforms.Compose([
                                    CenterCrop(),
                                    ToTensor(),
                                    Normalize()
                                ]))

In [32]:
def reverse_normalize(x, mean, std):
    x[:, 0, :, :] = x[:, 0, :, :] * std[0] + mean[0]
    x[:, 1, :, :] = x[:, 1, :, :] * std[1] + mean[1]
    x[:, 2, :, :] = x[:, 2, :, :] * std[2] + mean[2]
    return x

In [33]:
eval_loader = DataLoader(eval_data, batch_size=8, shuffle=False)

mean=[55.8630, 59.9099, 91.7419]
std=[31.6852, 29.8496, 19.0835]

for sample in eval_loader:
    trained_model.eval()
    
    predict(trained_model, sample)
    
    x = sample["image"]
    x = reverse_normalize(x, mean, std)
    save_image(x/255, "./FCN_results_without_class_weight/original_img_with_FCN.jpg")
    
    break